In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import OpenEXR

In [3]:
RADIUS = 4
SIDE = 1 + 2 * RADIUS

mask = np.zeros((SIDE, SIDE))
mask[RADIUS][RADIUS] = 30

def rotate_ij(direction, i, j):
  match direction:
    case 0: return (i,  j)
    case 1: return (j,  -i)
    case 2: return (-i, -j)
    case 3: return (-j, i)

for direction in range(4):
  for i in range(1, RADIUS+1):
    for j in range(-i, +i):
      ii, jj = rotate_ij(direction, i, j)
      mask[RADIUS + ii][RADIUS + jj] = 5 + direction * 6

px.imshow(mask)

In [4]:
def crop_exr(input_path, output_path, x, y, w, h):
  image = OpenEXR.File(input_path)
  header = image.header()
  wind = (np.array([0, 0], dtype=np.int32),
          np.array([w-1, h-1], dtype=np.int32))
  header['dataWindow'] = wind
  header['displayWindow'] = wind

  def crop_inner(c):
    c = c[y:y+h, x:x+w]
    return np.copy(c) # wouldn't work without copy, OpenEXR bug

  channels = {}
  for name, chan in image.channels().items():
    channels[name] = crop_inner(chan.pixels)

  with OpenEXR.File(header, channels) as outfile:
    outfile.write(output_path)

  #again = OpenEXR.File(output_path)
  #for name, chan in again.channels().items():
  #  display(chan)
  #  px.imshow(chan.pixels).show()

In [5]:
#crop_exr('../exr/bistro_cafe_gbuffer.exr',
#         '../exr/bistro_cropped.exr',
#         1300, 0, 500, 500)

In [ ]:
img = np.zeros((256,128))

for i in range(img.shape[0]):
  for j in range(img.shape[1]):
    ii = i // 11
    jj = j // 11
    img[i][j] = (ii + jj)

px.imshow(img).show()

imgf = img.flatten()
auxf = np.zeros_like(imgf)
width = img.shape[1]
total = imgf.shape[0]

for i in range(total):
  cx = (i & 0b0011)
  cy = (i & 0b1100) >> 2
  bid = i // 16
  by = bid // (width // 4)
  bx = bid % (width // 4)
  j = width * (4 * by + cy) + 4 * bx + cx
  auxf[i] += imgf[j]

def shift_origin(origin, dx, dy):
  cx = (origin & 0b0011)
  cy = (origin & 0b1100) >> 2
  bid = origin // 16
  cx += dx
  cy += dy
  if cx < 0:    bid -= 1
  elif cx >= 4: bid += 1
  if cy < 0:    bid -= width // 4
  elif cy >= 4: bid += width // 4
  assert bid >= 0
  cx += 4
  cy += 4
  cx &= 0b0011
  cy &= 0b0011
  return (bid << 4) | (cy << 2) | cx

blurf = np.empty_like(auxf)

redzone = 17 * width
for origin in range(redzone, total-redzone):
  value = 0
  for dx in range(-2, 2):
    for dy in range(-2, 2):
      offset = shift_origin(origin, dx, dy)
      value += auxf[offset] / 9.0
  blurf[origin] = value

for i in range(total):
  cx = (i & 0b0011)
  cy = (i & 0b1100) >> 2
  bid = i // 16
  by = bid // (width // 4)
  bx = bid % (width // 4)
  j = width * (4 * by + cy) + 4 * bx + cx
  imgf[j] += blurf[i]

px.imshow(imgf.reshape(img.shape)).show()